In [ ]:
#pip install findspark

  Using cached findspark-2.0.1-py2.py3-none-any.whl.metadata (352 bytes)
Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import findspark
findspark.init()
from pyspark.sql import functions as F


In [4]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Test_01")
    .config("spark.ui.port", "9311")
    .getOrCreate()
)

spark

25/12/09 11:59:10 WARN Utils: Your hostname, MacBook-Air-de-Irina.local resolves to a loopback address: 127.0.0.1; using 192.168.1.52 instead (on interface en0)
25/12/09 11:59:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/09 11:59:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/09 11:59:11 WARN Utils: Service 'SparkUI' could not bind on port 9311. Attempting port 9312.


In [6]:
df = spark.read.csv("Hotel.csv", header=True, inferSchema=True)
df.show()

+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
|      ID|n_adults|n_children|weekend_nights|week_nights|   meal_plan|car_parking_space|  room_type|lead_time|year|month|date|market_segment|repeated_guest|previous_cancellations|previous_bookings_not_canceled|avg_room_price|special_requests|      status|
+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
|INN00001|       2|         0|             1|          2| Meal Plan 1|                0|Room_Type 1|      224|2017|   10|   2|       Offline|             0|                     0|                             0|          65.0|       

In [7]:
spark.sql("""
    CREATE OR REPLACE TEMP VIEW calendar AS
    SELECT
        explode(sequence(
            to_date('2017-01-01'),
            to_date('2018-12-31'),
            interval 1 day
        )) AS calendar_dt
""")

DataFrame[]

In [8]:
spark.sql("SELECT * FROM calendar LIMIT 10").show()

+-----------+
|calendar_dt|
+-----------+
| 2017-01-01|
| 2017-01-02|
| 2017-01-03|
| 2017-01-04|
| 2017-01-05|
| 2017-01-06|
| 2017-01-07|
| 2017-01-08|
| 2017-01-09|
| 2017-01-10|
+-----------+



**1. Вычислить среднее количество ночей, которые гости проводят в отеле (только для подтвержденных бронирований, с детализацией по месяцам и годам)**

In [14]:
df.select("status").distinct().show(100)

+------------+
|      status|
+------------+
|Not_Canceled|
|    Canceled|
+------------+



In [16]:
result = (
    df.filter(F.col("status") == "Not_Canceled")
      .withColumn("total_nights", F.col("weekend_nights") + F.col("week_nights"))
      .groupBy("year", "month")
      .agg(F.round(F.avg("total_nights"), 2).alias("avg_nights"))
      .orderBy("year", "month")
)

result.show()

+----+-----+----------+
|year|month|avg_nights|
+----+-----+----------+
|2017|    7|      3.02|
|2017|    8|      2.72|
|2017|    9|      2.66|
|2017|   10|       2.7|
|2017|   11|      2.72|
|2017|   12|      3.04|
|2018|    1|      2.74|
|2018|    2|      2.69|
|2018|    3|      3.04|
|2018|    4|      2.92|
|2018|    5|      2.81|
|2018|    6|       2.6|
|2018|    7|      3.19|
|2018|    8|      3.15|
|2018|    9|      2.79|
|2018|   10|      2.89|
|2018|   11|      2.98|
|2018|   12|      3.25|
+----+-----+----------+



**2. Определить ТОП-3 месяца по проценту отмененных броней за 2018 год.**

In [19]:
result = (
    df.filter(F.col("year") == 2018)
      .groupBy("month")
      .agg(
          F.sum(F.when(F.col("status") == "Canceled", 1).otherwise(0)).alias("canceled_count"),
          F.count("*").alias("total_count")
      )
      .withColumn("cancel_percent", F.round(F.col("canceled_count") / F.col("total_count") * 100, 2))
      .orderBy(F.col("cancel_percent").desc())
      .limit(3)
)

result.show()

+-----+--------------+-----------+--------------+
|month|canceled_count|total_count|cancel_percent|
+-----+--------------+-----------+--------------+
|    8|          1303|       2799|         46.55|
|   10|          1578|       3404|         46.36|
|    9|          1356|       2962|         45.78|
+-----+--------------+-----------+--------------+



**3. Вычислить среднее время на каждый месяц между бронированием и заездом в отель для подтвержденных броней.**

In [20]:
result = (
    df.filter(F.col("status") == "Not_Canceled")
      .groupBy("year", "month")
      .agg(F.round(F.avg("lead_time"), 2).alias("avg_lead_time"))
      .orderBy("year", "month")
)

result.show()

+----+-----+-------------+
|year|month|avg_lead_time|
+----+-----+-------------+
|2017|    7|       130.73|
|2017|    8|        35.08|
|2017|    9|        51.72|
|2017|   10|        55.89|
|2017|   11|        33.28|
|2017|   12|        46.75|
|2018|    1|        34.87|
|2018|    2|        30.53|
|2018|    3|        43.19|
|2018|    4|        62.49|
|2018|    5|        60.99|
|2018|    6|        70.64|
|2018|    7|        86.88|
|2018|    8|        83.09|
|2018|    9|        63.32|
|2018|   10|        73.24|
|2018|   11|        44.25|
|2018|   12|        69.75|
+----+-----+-------------+



**4. Вычислить общую среднюю выручку на каждый месяц в каждом году, сгруппировав по всем типам бронирования для подтвержденных броней, и вывести это в виде сводной таблицы (PIVOT).**


In [23]:
df2 = (
    df.filter(F.col("status") == "Not_Canceled")
      .withColumn("total_nights", F.col("weekend_nights") + F.col("week_nights"))
      .withColumn("revenue", F.col("avg_room_price") * F.col("total_nights"))
)
df2.show()

+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+------------+------------------+
|      ID|n_adults|n_children|weekend_nights|week_nights|   meal_plan|car_parking_space|  room_type|lead_time|year|month|date|market_segment|repeated_guest|previous_cancellations|previous_bookings_not_canceled|avg_room_price|special_requests|      status|total_nights|           revenue|
+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+------------+------------------+
|INN00001|       2|         0|             1|          2| Meal Plan 1|                0|Room_Type 1|      224|2017|   10|   2|       Off

In [25]:
pivot_df = (
    df2.groupBy("year", "month")
       .pivot("market_segment")
       .agg(F.round(F.avg("revenue"), 2))
       .orderBy("year", "month")
)

pivot_df.show()

+----+-----+--------+-------------+---------+-------+------+
|year|month|Aviation|Complementary|Corporate|Offline|Online|
+----+-----+--------+-------------+---------+-------+------+
|2017|    7|    NULL|         22.4|   113.75| 228.95|290.56|
|2017|    8|    NULL|         0.32|   156.42| 235.54|284.21|
|2017|    9|    NULL|        16.89|   177.83| 236.65|348.55|
|2017|   10|    NULL|         1.09|   180.26| 223.24|311.47|
|2017|   11|    NULL|        14.81|   102.97| 198.36|240.52|
|2017|   12|    NULL|         0.25|   141.11| 253.86|258.93|
|2018|    1|    NULL|         2.27|   113.03| 210.51|236.09|
|2018|    2|   352.0|         1.39|   115.06| 251.85|238.07|
|2018|    3|  118.33|        38.17|   142.39| 233.39|301.71|
|2018|    4|  321.81|          0.0|   108.42| 236.44|320.08|
|2018|    5|   262.5|          0.0|    229.5| 274.55|352.34|
|2018|    6|   247.0|          0.0|   148.13| 251.98|335.03|
|2018|    7|    79.0|         5.38|   146.99| 310.36|390.05|
|2018|    8|    NULL|   

**5. Выявить ТОП-5 постоянных гостей, которые принесли наибольшую выручку за все время, и показать их долю в общей выручке от постоянных гостей. Использовать уникальный идентификатор брони как уникальный идентификатор гостя, предположив, что 1 бронь = 1 гость.**

Фильтруем постоянных гостей и считаем выручку

In [32]:
df_revenue = (
    df.filter(F.col("repeated_guest") == 1)
      .withColumn("total_nights", F.col("weekend_nights") + F.col("week_nights"))
      .withColumn("revenue", F.col("avg_room_price") * F.col("total_nights"))
)
df_revenue.show(5)

+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+------------+-------+
|      ID|n_adults|n_children|weekend_nights|week_nights|   meal_plan|car_parking_space|  room_type|lead_time|year|month|date|market_segment|repeated_guest|previous_cancellations|previous_bookings_not_canceled|avg_room_price|special_requests|      status|total_nights|revenue|
+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+------------+-------+
|INN00089|       2|         0|             0|          1|Not Selected|                0|Room_Type 1|       17|2018|   12|  10|        Online|             1|             

Считаем суммарную выручку на каждого гостя

In [33]:
guest_revenue = (
    df_revenue.groupBy("ID")
              .agg(F.sum("revenue").alias("total_revenue"))
)
guest_revenue.show(5)

+--------+-------------+
|      ID|total_revenue|
+--------+-------------+
|INN00606|        130.0|
|INN03841|         55.6|
|INN07524|        201.0|
|INN15032|        195.0|
|INN23331|         66.0|
+--------+-------------+
only showing top 5 rows



 Находим ТОП-5 гостей

In [28]:
top5 = guest_revenue.orderBy(F.col("total_revenue").desc()).limit(5)
top5.show()

+--------+------------------+
|      ID|     total_revenue|
+--------+------------------+
|INN19235|1754.3999999999999|
|INN05222|             690.0|
|INN14189|             665.0|
|INN09923|             660.0|
|INN25479|             650.0|
+--------+------------------+



Считаем общую выручку всех постоянных гостей

In [30]:
total_revenue_all = guest_revenue.agg(F.sum("total_revenue").alias("total")).first()["total"]
total_revenue_all

116004.47999999997

Добавляем долю каждого из ТОП-5 гостей

In [31]:
top5_with_share = (
    top5.withColumn("share_percent", F.round(F.col("total_revenue") / total_revenue_all * 100, 2))
)

top5_with_share.show()

+--------+------------------+-------------+
|      ID|     total_revenue|share_percent|
+--------+------------------+-------------+
|INN19235|1754.3999999999999|         1.51|
|INN05222|             690.0|         0.59|
|INN14189|             665.0|         0.57|
|INN09923|             660.0|         0.57|
|INN25479|             650.0|         0.56|
+--------+------------------+-------------+



**6.Вывести общее количество гостей на каждый день в отеле, отсортировав по убыванию дат, включая дни, когда отель пустует. Также рассчитать процент загрузки для каждого дня, если известно, что общая вместимость отеля 400 человек.**

Создаём поле даты прибытия и даты выезда

In [36]:
df2 = (
    df.withColumn("arrival_date",
                  F.to_date(F.concat_ws("-", "year", "month", "date")))
      .withColumn("stay_nights",
                  F.col("weekend_nights") + F.col("week_nights"))
      .withColumn("departure_date",
                  F.expr("date_add(arrival_date, stay_nights)"))
)
df2.show(5)

+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+------------+-----------+--------------+
|      ID|n_adults|n_children|weekend_nights|week_nights|   meal_plan|car_parking_space|  room_type|lead_time|year|month|date|market_segment|repeated_guest|previous_cancellations|previous_bookings_not_canceled|avg_room_price|special_requests|      status|arrival_date|stay_nights|departure_date|
+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+------------+-----------+--------------+
|INN00001|       2|         0|             1|          2| Meal Plan 1|                0|Room_Type 1|      224|20

Джойним с календарем. Вьюху нужно перевести в датафрейм.

In [42]:
calendar_df = spark.table("calendar")

stay_df = (
    df2.join(
        calendar_df,
        (F.col("calendar_dt") >= F.col("arrival_date")) &
        (F.col("calendar_dt") < F.col("departure_date")),
        "left"
    )
)

stay_df.show(5)

+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+------------+-----------+--------------+-----------+
|      ID|n_adults|n_children|weekend_nights|week_nights|   meal_plan|car_parking_space|  room_type|lead_time|year|month|date|market_segment|repeated_guest|previous_cancellations|previous_bookings_not_canceled|avg_room_price|special_requests|      status|arrival_date|stay_nights|departure_date|calendar_dt|
+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+------------+-----------+--------------+-----------+
|INN00001|       2|         0|             1|          2| Meal Plan 1|      

Считаем гостей по каждому дню и добавляем % загрузки

In [50]:
daily_guests = (
    stay_df.groupBy("calendar_dt")
           .agg(F.count("*").alias("guest_count"))
           .withColumn("occupancy_percent", F.round(F.col("guest_count") / 400 * 100, 2))
           .orderBy(F.col("calendar_dt").desc())
)
daily_guests.show()

+-----------+-----------+-----------------+
|calendar_dt|guest_count|occupancy_percent|
+-----------+-----------+-----------------+
| 2018-12-31|        365|            91.25|
| 2018-12-30|        375|            93.75|
| 2018-12-29|        377|            94.25|
| 2018-12-28|        342|             85.5|
| 2018-12-27|        344|             86.0|
| 2018-12-26|        263|            65.75|
| 2018-12-25|        216|             54.0|
| 2018-12-24|        201|            50.25|
| 2018-12-23|        180|             45.0|
| 2018-12-22|        157|            39.25|
| 2018-12-21|        148|             37.0|
| 2018-12-20|        145|            36.25|
| 2018-12-19|        144|             36.0|
| 2018-12-18|        160|             40.0|
| 2018-12-17|        172|             43.0|
| 2018-12-16|        168|             42.0|
| 2018-12-15|        109|            27.25|
| 2018-12-14|        102|             25.5|
| 2018-12-13|        102|             25.5|
| 2018-12-12|        109|       